# Crawling: Controlled

In [1]:
import time
import re


from requests_html import HTMLSession
from bs4 import BeautifulSoup as bs
from fuzzywuzzy import fuzz

In [2]:
session = HTMLSession()

In [10]:
# Start with this gather links and then comments
url = 'https://www.kino.dk/film/m/mi/mission-impossible-6-fallout'
pattern = re.compile('/film/\w+/\w{1,}/')

In [4]:
r = session.get(url)

### Deep Scrap and Crawls

In [11]:
links, used = set(),set()
def gather_url(url):
    print((len(links),len(used)), end=',')
    k = session.get(url)
    used.add(url)
    g_links = set([i for i in k.html.links if (re.match(pattern,i) and '#' not in i)])
    
    for link in g_links:
        if link not in links:
            links.add(link)

            if link not in used and 'page=' not in link:
                gather_url(f'https://www.kino.dk{link}')
                time.sleep(2)

    return links
    

In [12]:
links = gather_url(url)

(0, 0),(1, 1),(2, 2),(3, 3),(4, 4),(5, 5),(6, 6),(7, 7),(8, 8),(9, 9),(10, 10),(11, 11),(13, 12),(14, 13),(15, 14),(16, 15),(18, 16),(19, 17),(20, 18),(21, 19),(22, 20),(23, 21),(24, 22),

KeyboardInterrupt: 

In [13]:
links

{'/film/a/av/avengers-4',
 '/film/b/br/britt-marie-var-her',
 '/film/d/di/diego-maradona',
 '/film/d/do/downton-abbey',
 '/film/d/dr/dronningen',
 '/film/d/dr/dronningen?page=4',
 '/film/f/fi/five-feet-apart',
 '/film/g/go/goeg-og-gokke',
 '/film/h/hu/hustlers',
 '/film/i/it/it-del-2',
 '/film/j/jo/john-wick-3-parabellum',
 '/film/j/jo/john-wick-3-parabellum?page=1',
 '/film/k/kv/kvinde-paa-krigsstien',
 '/film/k/kv/kvinden-der-glemte-sin-fortid',
 '/film/l/lo/lovernes-konge',
 '/film/m/me/men-black-international',
 '/film/m/mr/mrs-hyde-0',
 '/film/o/on/once-upon-time-hollywood',
 '/film/p/pu/push-0',
 '/film/r/ra/rambo-5-last-blood',
 '/film/s/so/sommer',
 '/film/t/th/conjuring-3',
 '/film/t/th/darkest-minds',
 '/film/v/va/van-gogh-ved-evighedens-port'}

In [14]:
def get_soups(url):
    r = session.get(url)
    srcs = []
    for link in r.html.absolute_links:
        ratio = fuzz.token_sort_ratio(url.split('/',3)[-1], link) 
        if ratio > 70:
            print(f'\nMatch: {ratio}. Requesting {link}')
            req = session.get(link)
            if req.ok:
                print('Preparing our soup')
                srcs.append(bs(req.content,'lxml'))
            else:
                raise ConnectionError('Connection Failed')
    return srcs

In [15]:
srcs = get_soups(url=url)


Match: 75. Requesting https://www.kino.dk/film/m/mi/mission-impossible-fallout?page=2
Preparing our soup

Match: 75. Requesting https://www.kino.dk/film/m/mi/mission-impossible-fallout?page=3
Preparing our soup

Match: 75. Requesting https://www.kino.dk/film/m/mi/mission-impossible-fallout?page=1
Preparing our soup

Match: 75. Requesting https://www.kino.dk/film/m/mi/mission-impossible-fallout?page=4
Preparing our soup

Match: 80. Requesting https://www.kino.dk/film/m/mi/mission-impossible-fallout
Preparing our soup


In [16]:
srcs[0]

<!DOCTYPE html>
<html lang="da" xmlns:fb="http://ogp.me/ns/fb#" xmlns:og="http://opengraphprotocol.org/schema/">
<head>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<title>Mission: Impossible - Fallout</title>
<!--[if IE]><![endif]-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script async="async" defer="defer" src="https://www.google.com/recaptcha/api.js?hl=da"></script>
<style>.async-hide { opacity: 0 !important} </style>
<script>(function(a,s,y,n,c,h,i,d,e){s.className+=' '+y;h.start=1*new Date;
h.end=i=function(){s.className=s.className.replace(RegExp(' ?'+y),'')};
(a[n]=a[n]||[]).hide=h;setTimeout(function(){i();h.end=null},c);h.timeout=c;
})(window,document.documentElement,'async-hide','dataLayer',4000,
{'GTM-W9Z3HGR':true});</script>
<!-- MODIFIED ANALYTICS TRACKING CODE WITH OPTIMIZE PLUGIN: -->
<!-- REMEMBER TO INSERT CORRECT ID’S FOR GA AND GO: -->
<script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i

In [17]:
comments = srcs[0].find_all('a',{'id':re.compile('comment-\d{2,}')})

In [18]:
len(comments)

10

In [19]:
#Post Time
data = [i.next_sibling.next_sibling for i in comments]
for i in data:
    x = i.find('div',{'class':'submitted'})
    print(x.span['content'])#('content')

2018-08-14T12:54:08+02:00
2018-08-12T23:05:04+02:00
2018-08-11T23:18:44+02:00
2018-08-10T13:06:38+02:00
2018-08-09T21:21:15+02:00
2018-08-08T21:58:50+02:00
2018-08-11T02:44:44+02:00
2018-08-13T18:47:08+02:00
2018-08-07T19:50:45+02:00
2018-08-07T12:08:12+02:00


In [20]:
#Post UserID
data = [i.next_sibling.next_sibling for i in comments]
for i in data:
    x = i.a['href'].split('/')[-1]
    print(x)#('content')

12928
1235159
1502933
17123
523184
507172
482401
507172
11915
479396


In [21]:
#Post CommentID
data = [i.next_sibling.next_sibling for i in comments]
for i in data:
    if i.find('div',{'class','user-review-vote'}):
        x = i.h3.a['href'].split('-')[-1]
        print(x)

246275
246204
246173
246159
246133
246109
246098


In [22]:
#Post Comments
data = [i.next_sibling.next_sibling for i in comments]
for i in data:
    if i.find('div',{'class','user-review-vote'}):
        x = i.find('div',{'class':'content'})
        print(x.get_text())
        print('-'*10)


Helt som den skal være, fuld fart frem, små overraskende momenter, og stadig god humor. Man får en god historie som altid og et smil på læben. Samtidig tænker man lige, hvilke stunts er det lige Tom selv laver her!
Dog var lyden lige skruet højt nok op i salen med så få gæster!
 
----------

Da jeg var inde for at se Mission Impossible 6 gik jeg efter at se preikestolen i Norge men blev skuffet da der stod det var i karsahkstan da jeg selv har været på preikestolen. Og for dem der ikke kender preikestolen men som har set filmen så er det den klippe de hænger på og hvor ethan Hunt detonere atombomberne.
 
----------

Nøjagtig som med de øvrige MI film - bliver du lynhurtigt kastet ind i det her agent set-up
Super spændende fra ende til anden - og med lidt kant af romantik - herligt
 
----------

Til folk der er fans af Simon pegg, prøv at checke hans nye film ud han kommer med kaldet slaughterhouse rulez, ser dæleme sjov ud
 
----------

Egentlig ser jeg Mission Impossible pga. Simon P

In [23]:
#Post Score
data = [i.next_sibling.next_sibling for i in comments]
for i in data:
    if i.find('div',{'class','user-review-vote'}):
        x =i.find('div',{'class','user-review-vote'}).div.div['title']
        print(x)
        print('-'*10)

5
----------
5
----------
5
----------
5
----------
6
----------
5
----------
5
----------


In [24]:
#Post Score
data = [i.next_sibling.next_sibling for i in comments]
for i in data:
    if i.find('div',{'class','user-review-vote'}):
        commentdate = i.find('div',{'class':'submitted'}).span['content']
        userid = i.a['href'].split('/')[-1]
        commentid = i.h3.a['href'].split('-')[-1]
        comment = i.find('div',{'class':'content'}).get_text()
        score =i.find('div',{'class','user-review-vote'}).div.div['title']
        print(commentid,score, comment)
        print('-'*10)

246275 5 
Helt som den skal være, fuld fart frem, små overraskende momenter, og stadig god humor. Man får en god historie som altid og et smil på læben. Samtidig tænker man lige, hvilke stunts er det lige Tom selv laver her!
Dog var lyden lige skruet højt nok op i salen med så få gæster!
 
----------
246204 5 
Da jeg var inde for at se Mission Impossible 6 gik jeg efter at se preikestolen i Norge men blev skuffet da der stod det var i karsahkstan da jeg selv har været på preikestolen. Og for dem der ikke kender preikestolen men som har set filmen så er det den klippe de hænger på og hvor ethan Hunt detonere atombomberne.
 
----------
246173 5 
Nøjagtig som med de øvrige MI film - bliver du lynhurtigt kastet ind i det her agent set-up
Super spændende fra ende til anden - og med lidt kant af romantik - herligt
 
----------
246159 5 
Til folk der er fans af Simon pegg, prøv at checke hans nye film ud han kommer med kaldet slaughterhouse rulez, ser dæleme sjov ud
 
----------
246133 6 
Ege

In [25]:
comment_l = []
def comment_data(src):
    for k, j in enumerate(src):
        data = [i.next_sibling.next_sibling for i in srcs[k].find_all('a',{'id':re.compile('comment-\d{2,}')})]
        for i in data:
            if i.find('div',{'class','user-review-vote'}):
                commentdate = i.find('div',{'class':'submitted'}).span['content']
                userid = i.a['href'].split('/')[-1]
                commentid = i.h3.a['href'].split('-')[-1]
                comment = i.find('div',{'class':'content'}).get_text()
                score =i.find('div',{'class','user-review-vote'}).div.div['title']
                comment_l.append([commentdate, userid, commentid, comment, int(score)])
    return comment_l

In [26]:
a = comment_data(srcs)

In [27]:
a

[['2018-08-12T23:05:04+02:00',
  '1235159',
  '246275',
  '\nHelt som den skal være, fuld fart frem, små overraskende momenter, og stadig god humor. Man får en god historie som altid og et smil på læben. Samtidig tænker man lige, hvilke stunts er det lige Tom selv laver her!\nDog var lyden lige skruet højt nok op i salen med så få gæster!\n ',
  5],
 ['2018-08-11T23:18:44+02:00',
  '1502933',
  '246204',
  '\nDa jeg var inde for at se Mission Impossible 6 gik jeg efter at se preikestolen i Norge men blev skuffet da der stod det var i karsahkstan da jeg selv har været på preikestolen. Og for dem der ikke kender preikestolen men som har set filmen så er det den klippe de hænger på og hvor ethan Hunt detonere atombomberne.\n ',
  5],
 ['2018-08-10T13:06:38+02:00',
  '17123',
  '246173',
  '\nNøjagtig som med de øvrige MI film - bliver du lynhurtigt kastet ind i det her agent set-up\nSuper spændende fra ende til anden - og med lidt kant af romantik - herligt\n ',
  5],
 ['2018-08-09T21:21:

In [28]:
import pandas as pd

In [29]:
df  = pd.DataFrame(a, columns='reviewdt commentid userid review rate'.split())

In [30]:
df['review'].apply(str.strip)[2]

'Nøjagtig som med de øvrige MI film - bliver du lynhurtigt kastet ind i det her agent set-up\nSuper spændende fra ende til anden - og med lidt kant af romantik - herligt'

In [31]:
df

,reviewdt,commentid,userid,review,rate
0,2018-08-12T23:05:04+02:00,1235159,246275,"\nHelt som den skal være, fuld fart frem, små ...",5
1,2018-08-11T23:18:44+02:00,1502933,246204,\nDa jeg var inde for at se Mission Impossible...,5
2,2018-08-10T13:06:38+02:00,17123,246173,\nNøjagtig som med de øvrige MI film - bliver ...,5
3,2018-08-09T21:21:15+02:00,523184,246159,"\nTil folk der er fans af Simon pegg, prøv at ...",5
4,2018-08-08T21:58:50+02:00,507172,246133,\nEgentlig ser jeg Mission Impossible pga. Sim...,6
5,2018-08-07T19:50:45+02:00,11915,246109,\nDer var ikke meget tid til at rette dig op i...,5
6,2018-08-07T12:08:12+02:00,479396,246098,"\nMan er sjældent skuffet, når Tom Cruise optr...",5
7,2018-08-06T02:26:50+02:00,8516,246090,"\nEnergisk, følger stilen - god humor\nindimel...",5
8,2018-08-05T18:01:48+02:00,12930,246086,\nTom Cruise & Co. leverer endnu en fantastisk...,5
9,2018-08-04T09:57:16+02:00,9368,246073,\nMission Impossible Fallout er den 6. i række...,3


In [ ]:
for i in range(9):
    print(df['review'].apply(str.strip)[i])
    print('-'*15)